In [3]:
import pandas as pd
from nltk.stem import *
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
import spacy
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from currency_converter import CurrencyConverter
import re 
import string
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/damianzeller/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/damianzeller/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
pip install --upgrade scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [5]:
df = pd.read_csv("/Users/damianzeller/Desktop/HS23/ADM/dataframe_HW3.csv")
df['description'] = df['description'].str.replace('About the course', '')

2.0.0 Processing the text

In [6]:
#Function for general processing
def processing(x):
    #make sure that input is a string and case insensitive
    words = nltk.word_tokenize(str(x).lower())
    #stopwords, punctuation and nonnumerical
    lst_stopwords = stopwords.words('english')
    puncto = string.punctuation
    text_preproc= [word for word in words if not word in lst_stopwords and word.isalpha() and word not in puncto]
    #return the processed text as string
    return ' '.join(text_preproc)

#Delete the rows with no description
df = df[df.description != '']

#Apply the function to the description column and store result in desc_preproc
df['desc_preproc'] = df['description'].apply(processing)

#Stemming
stemmer = PorterStemmer()
df['stemmo_clean'] = df.desc_preproc.apply(lambda row: [stemmer.stem(word) for word in row.split(' ')])

2.0.1 Preprocessing the fees column

First we want to extract the currency, we will do this by applying a function and storing the result in a new column called currency

In [8]:
def extract_currency(currency):
    # Exception handling: Check if input is a string or if there is 
    if isinstance(currency, str):
        # Find all currency symbols in the string
        symbols = re.findall(r'[£$€¥¢]', currency)
        # Return the first symbol found, or None if no symbols are found,also handels empty strings
        return symbols[0] if symbols else None
    else:
        # Handle non-string inputs
        return None
#Applying the function to all rows in the dataframe and store the result in new column 'currency' 
df['currency'] = df['fees'].apply(extract_currency)

In a second step we extract the highest fee and store the result in a new column highest_fee

In [11]:
def extract_highest_fee(fee):
    if isinstance(fee, list):
        # Exception handling: If the fee is a list, convert each item to a string and apply the function recursively
        return max(extract_highest_fee(item) for item in fee) if fee else None
    elif isinstance(fee, str):
        # If the fee is a string, we just find all numbers and select the highest
        numbers = re.findall(r'\d+', fee)
        numbers = [int(num) for num in numbers]
        return max(numbers) if numbers else None
    elif isinstance(fee, float):
        # If the fee is a float, we don't have to do anything
        return fee
    else:
        raise TypeError(f"Unsupported type: {type(fee)}")

df['highest_fee'] = df['fees'].apply(extract_highest_fee)

In [12]:
which_currency = df['currency'].value_counts()
print(which_currency)

£    1102
€     283
$      32
Name: currency, dtype: int64


There are only three different currencies, we decide to convert them to euros

In [14]:
def convert_to_eur(row):
    #Write a currency mapping in order to make them convertible
    currency_mapping = {
    '$': 'USD',
    '€': 'EUR',
    '£': 'GBP',}
    #Handling the case, that there was no currency found before
    if pd.isnull(row['currency']):
        return None
    #Fees that we actually have to convert: Either Dollar or British Pound
    elif row['currency'] != 'EUR':
        cc = CurrencyConverter()
        currency_code = currency_mapping.get(row['currency'])
        if currency_code is not None:
            currency_value = cc.convert(1, currency_code, 'EUR') * row['highest_fee']
            return currency_value
        #Exception handling: 
        else:
            raise ValueError(f"{row['currency']} is not a supported currency")
    #Currency is already Euros, we don't have to do anything        
    else:
        return row['highest_fee']

#Apply the function to all rows in the dataframe and store it in the new column 'fees_in_eur'
df['fees_in_eur'] = df.apply(convert_to_eur, axis=1)

Now we have everything we need in the column 'fees_in_eur'. We delete the old 'fees' column and rename the column 'fees_in_eur' to 'fees'. Further we delete the columns 'currency' and 'highest_fee' as we don't need them anymore. 

In [15]:
#Delete the old 'fees' column
df.drop(['fees'], axis=1, inplace=True)

#Rename the 'fees_in_eur' column
df.rename(columns={'fees_in_eur': 'fees'}, inplace=True)

#Delete the 'highest_fee' and the 'currency' column
df.drop(['highest_fee', 'currency'], axis=1, inplace=True)

We print the first 100 rows of the dataframe and our results seem reasonable

In [ ]:
df.head(100)

2.1 

2.1.1

First we build the vocabulary

In [ ]:
#Initialize the vocabulary
vocabulary = set()

#Put the elements of 'stemmo_clean' column into the vocabulary
df.stemmo_clean.apply(lambda row: [vocabulary.add(word) for word in row])
vocabulary

Now we assign an ID to each word in the vocabulary. We do this by creating a dictionnary called index

In [ ]:
#Initialize a dictionnary
index = {}

#We make a list out of the vocabulary in order to loop over it
vocabulary = list(vocabulary) 

#Initialize first ID
unique_id = 1

#Loop over the vocabulary
for word in vocabulary:
    index[unique_id] = word
    unique_id+=1
index

We create the inverted index. It's a dictionnary called 'inverted_index'

In [20]:
inverted_index = {}

#Function to return True or False if the word is there
def isthere(x, value):
    if value in x:
        return True
    return False

#Loop over the 'index dictionnary
for key, value in index.items():
    #Initialize a list to store all the row numbers where to word is present
    listo = []
    #Loop over the dataframe
    for i, row in df.iterrows():
        #Function is called and adds every row where the word is present to the list
        if isthere(row['stemmo_clean'], value):
            listo.append(i)
    #When we looped over all rows the list with the row numbers, the list is being assigned as value ro to correspoinding ID of the word
    inverted_index[key] = listo

In [ ]:
inverted_index

We create our first search engine. We do this by writing a fucntion that is called search

In [21]:
def search(query):
    #We split the input into single words and initialize a list
    query_words = query.split()
    document_lists = []
    #Loop over the individual words
    for word in query_words:
        #As the words in our documents are preprocessed and stemmed we have to the same with our query
        stemmed_word = stemmer.stem(word)
        #We check if the word shows up in our documents and was assigned an ID
        if stemmed_word in index.values():
            #Loop over our 'index' dictionnary
            for key, value in index.items():
                if value == stemmed_word:
                    #We add the the list of documents where the word showed up to the 'document_lists'
                    document_lists.append(inverted_index[key])
                    break
        else:
            print(f"Unfortunately your search was not successful.")
    #If the search was successful we compute the intersection of the lists(as all the words must show up in the document)
    if document_lists:
        common_documents = set(document_lists[0]).intersection(*document_lists)
        #Create the dataframe as requested in the task
        new_df = df.loc[common_documents, ['courseName', 'universityName', 'description', 'url']].copy()
        return new_df
    #If there is no intersection between the lists of documents the search was not successfull
    else:
        print(f"Unfortunately your search was not successful.")

Now we check if our search engine works with the query used as an example in the task. The results we get seem reasonable

In [ ]:
search('advanced knowledge')

2.2

First we calculate our second inverted index, we call it 'inverted_index_due'

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transform our 'stemmo_clean' column to a list of strings. Each string is a document
documents = [' '.join(words) for words in df['stemmo_clean'].tolist()]

#Process data to be able to calculate the TFIDF
matrix = tfidf_vectorizer.fit_transform(documents)

# Get feature names 
transformed_words = tfidf_vectorizer.get_feature_names_out()

# Initialize the dictionnary
inverted_index_due = {}

# Loop over all the words
for index, transformed_word in enumerate(transformed_words):
    #Create the dictionnary key
    inverted_index_due[transformed_word] = []
    #Loop over all the rows
    for row in range(len(df)):  
        #Create tfidf score
        tfidf_score = matrix[row, index]
        #Documents with a tfidf of are left out
        if tfidf_score > 0:
            #Assign the tuple as value
            inverted_index_due[transformed_word].append((df.index[index], tfidf_score))

In [ ]:
inverted_index_due

In [ ]:
1. Solve the about the course
2. Proper preprocessing
3. Clean dataframe: Make a string out of the list
4. Preprocess inputqueries
5. TSV file
6. Save all the things that take long to compute in a file



After we have created our inverted index, we create our second serach engine, we call it how_similar

In [29]:
def how_similar(query):
    
    #We preprocess our input quety
    query= processing(query)
    words = word_tokenize(query)
    stemmed_words = [stemmer.stem(word) for word in words]
    stemmed_query = ' '.join(stemmed_words)
    
    # Make TFIDF representation of our query
    query_tfidf = tfidf_vectorizer.transform([stemmed_query])
    
    # Calculate cosine similarities of the query with the documents
    similarities = cosine_similarity(query_tfidf,matrix)
    similarities = similarities.flatten()
    
    #We create the dataframe requested in the task
    result_df= pd.DataFrame({
    'courseName': df['courseName'],
    'universityName': df['universityName'],
    'description': df['description'],
    'url': df['url'],
    'similarity_score': similarities})
    
    #Sort, so that the most similar documents are shown first
    result_df = result_df.sort_values(by='similarity_score', ascending=False)
    return result_df

In [ ]:
how_similar('advanced knowledge')

We test our search engine with some input and the results seem reasonable.